# MAPF Demo: Multi-Agent Path Finding

This notebook demonstrates MAPF (Multi-Agent Path Finding) using the PIBT algorithm. We'll solve examples on both random and warehouse maps with interactive visualizations.

## 1. Import Libraries

In [ ]:
from pypibt import PIBT, get_grid, get_scenario, is_valid_mapf_solution, save_configs_for_visualizer

## 2. Load MAPF Instance

Load a MAPF problem with:
- Map file (grid layout)
- Scenario file (start and goal positions)
- Number of agents

In [ ]:
# Load problem instance
grid = get_grid("../assets/maps/random-32-32-10.map")
starts, goals = get_scenario("../assets/mapf/random-32-32-10-random-1.scen", N=100)

print(f"MAPF Instance:")
print(f"  Grid size: {grid.shape}")
print(f"  Number of agents: {len(starts)}")

## 3. Solve the Problem

Run the PIBT solver to find collision-free paths.

In [ ]:
# Solve MAPF
import time
start_time = time.perf_counter()
pibt = PIBT(grid, starts, goals, seed=42)
solution = pibt.run(max_timestep=1000)
comp_time = time.perf_counter() - start_time

print(f"Solution completed!")
print(f"Timesteps: {len(solution) - 1}")
print(f"Computation time: {comp_time:.2f}s")

## 4. Validate Solution

In [ ]:
# Validate the solution
valid = is_valid_mapf_solution(grid, starts, goals, solution)
print(f"Valid solution: {valid}")

## 5. Save Solution

In [ ]:
# Save solution
output_file = "../assets/mapf/output_mapf.txt"
save_configs_for_visualizer(solution, output_file)
print(f"Solution saved to {output_file}")

## 6. Visualize Solution

Use the Python visualizer to create an animated video showing the agents executing their paths.

In [ ]:
import sys
sys.path.append('..')

from scripts.visualize_mapd import MAPDVisualizer, load_solution, get_grid as viz_get_grid
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend for notebook
import matplotlib.pyplot as plt
from IPython.display import Image
from matplotlib.animation import PillowWriter

# Load the solution
grid_viz = viz_get_grid("../assets/maps/random-32-32-10.map")
solution_viz, agent_goals_viz = load_solution("../assets/mapf/output_mapf.txt")

print(f"Loaded solution: {len(solution_viz)} timesteps, {len(solution_viz[0])} agents")

# Create visualizer with interpolation for smooth animation
visualizer = MAPDVisualizer(grid_viz, solution_viz, agent_goals=agent_goals_viz, 
                            interpolation_steps=5, mode='mapf')

# Create animation and save as GIF (works without ffmpeg)
fig, anim = visualizer.animate(interval=20)

# Save as GIF using Pillow (no ffmpeg required)
anim.save("../assets/mapf/output_mapf.gif", writer=PillowWriter(fps=50))
plt.close(fig)

# Display the GIF in notebook
Image(filename="../assets/mapf/output_mapf.gif")

## 7. Warehouse Map Example

Now let's try a larger warehouse map with more agents.

In [ ]:
# Load warehouse map and scenario
warehouse_grid = get_grid("../assets/maps/warehouse-20-40-10-2-2.map")
warehouse_starts, warehouse_goals = get_scenario("../assets/mapf/warehouse-20-40-10-2-2-random-1.scen", N=20)

# Solve
warehouse_pibt = PIBT(warehouse_grid, warehouse_starts, warehouse_goals, seed=42)
warehouse_solution = warehouse_pibt.run(max_timestep=2000)

# Validate
warehouse_valid = is_valid_mapf_solution(warehouse_grid, warehouse_starts, warehouse_goals, warehouse_solution)

print(f"Warehouse Results:")
print(f"  Grid: {warehouse_grid.shape}")
print(f"  Agents: {len(warehouse_starts)}")
print(f"  Valid: {warehouse_valid}")
print(f"  Timesteps: {len(warehouse_solution) - 1}")

## Summary

This notebook demonstrated:
- Loading MAPF instances with map and scenario files
- Solving with PIBT algorithm for collision-free paths
- Validating solutions
- Visualizing agent behavior

**Visualization Features:**
- **Smooth interpolation**: Agents move fluidly between timesteps
- **Goal lines**: Lines from each agent to their final goal position
- **GIF animations**: Animations saved and displayed as GIF images (no ffmpeg required)
- **Export options**: Animations automatically saved as GIF files
- **Format compatibility**: Works with both MAPF and MAPD output formats, MAPF is also compatible with the external [mapf-visualizer](https://github.com/kei18/mapf-visualizer) tool

**Key Parameters:**
- `map_file`: Path to map file
- `scen_file`: Path to scenario file with start/goal positions
- `N`: Number of agents
- `seed`: Random seed for reproducibility
- `max_timestep`: Maximum simulation steps
- `interpolation_steps`: Frames per timestep for smooth animation (default: 5)
- `fps`: Frames per second for GIF export (default: 50)
- `mode`: Visualization mode - set to 'mapf' to show final goals